In [ ]:
# I. Logistic regression on a single SNP under an allelic model
plink --ped ./data/dbp.cc.ped --map ./data/dbp.map --map3 --out logreg.add  --logistic --noweb
cat logreg.add.assoc.logistic

plink --ped ./data/dbp.cc.ped --map ./data/dbp.map --map3 --out logreg.add.ci  --logistic  --ci 0.95  --noweb
cat logreg.add.ci.assoc.logistic

In [ ]:
pwd

In [ ]:
# II. Adjustment for the effects of covariates and of other SNPs
plink --ped dbp.cc.ped --map dbp.map --map3 --out logreg.age.add  --logistic  --covar dbp.age.pheno  --noweb
cat logreg.age.add.assoc.logistic

plink --ped dbp.cc.ped --map dbp.map --map3 --out logreg.sex.add  --logistic --sex  --noweb
cat logreg.sex.add.assoc.logistic

plink --ped dbp.cc.ped --map dbp.map --map3 --out logreg.sexage.add  --logistic --sex  --covar dbp.age.pheno  --noweb
cat logreg.sexage.add.assoc.logistic

plink --ped dbp.cc.ped --map dbp.map --map3 --out logreg.snp1112.add  --logistic  --condition rs1112  --noweb
cat logreg.snp1112.add.assoc.logistic

plink --ped dbp.cc.ped --map dbp.map --map3 --out logreg.snp1117.add  --logistic  --condition rs1117
cat logreg.snp1117.add.assoc.logistic

In [ ]:
# III. Analysis of quantitative instead of dichotomized trait
plink --ped dbp.qt.ped --map dbp.map --map3 --out linreg.sex.add  --linear   --sex  --noweb
cat linreg.sex.add.assoc.linear

In [ ]:
# IV. Gene-environment (GxE) and gene-gene (GxG) interaction
plink --ped dbp.cc.ped --map dbp.map --map3 --out logreg.sex.inter.add  --logistic  --sex  --interaction  --noweb
cat logreg.sex.inter.add.assoc.logistic

plink --ped dbp.cc.ped --map dbp.map --map3 --out logreg.snp1112.inter.add  --logistic  --condition rs1112  --interaction  --noweb
cat logreg.snp1112.inter.add.assoc.logistic

In [3]:
load ("./data/dbp.R")
ls ()
dbp[1:5,]

summary(dbp)

[1] "dbp"

fam  pid fid mid sex trait affection age rs1101 rs1102 ⋯ rs1111 rs1112
4928 4928 1   0   0   1   85.51 1         66  4      2      ⋯ 3      3     
1838 1838 1   0   0   1   84.51 1         67  2      2      ⋯ 4      2     
2450 2450 1   0   0   1   84.30 1         89  2      2      ⋯ 4      3     
647   647 1   0   0   2   89.14 1         36  4      4      ⋯ 2      3     
2772 2772 1   0   0   1   90.39 1         54  3      2      ⋯ 2      2     
     rs1113 rs1114 rs1115 rs1116 rs1117 rs1118 rs1119 rs1120
4928 3      3      3      4      2      3      3      3     
1838 2      4      4      4      2      2      4      4     
2450 3      4      3      4      2      3      3      3     
647  3      2      3      4      3      3      4      3     
2772 2      2      4      4      2      2      4      4

      fam            pid         fid         mid    sex         trait       
 Min.   :   1   Min.   :1   Min.   :0   Min.   :0   1:329   Min.   : 60.50  
 1st Qu.:1250   1st Qu.:1   1st Qu.:0   1st Qu.:0   2:271   1st Qu.: 77.44  
 Median :2600   Median :1   Median :0   Median :0           Median : 82.00  
 Mean   :2536   Mean   :1   Mean   :0   Mean   :0           Mean   : 81.85  
 3rd Qu.:3729   3rd Qu.:1   3rd Qu.:0   3rd Qu.:0           3rd Qu.: 86.09  
 Max.   :5000   Max.   :1   Max.   :0   Max.   :0           Max.   :101.49  
 affection      age        rs1101  rs1102  rs1103   rs1104    rs1105 
 0:300     Min.   :18.00   2:115   2:317   2:165   2   : 32   2:118  
 1:300     1st Qu.:38.00   3:311   3:249   3:309   3   :208   3:311  
           Median :55.00   4:174   4: 34   4:126   4   :268   4:171  
           Mean   :55.49                           NA's: 92          
           3rd Qu.:74.00                                             
           Max.   :90.00                 

In [4]:
# --- I. Logistic regression on a single SNP genotype --- #

# Regression + Wald test #
result.snp12 = glm (affection ~ rs1112, family=binomial("logit"), data=dbp)
print ( result.snp12 )
print ( class (result.snp12) )
print ( summary(result.snp12) )

# LRT #
dev.geno = anova (result.snp12, test="Chi")
  lrt.pvalue = pchisq(dev.geno[dim(dev.geno)[1],"Deviance"], df=2, ncp=0, FALSE) 
  print ( lrt.pvalue )

# OR + CI #
print ( summary(result.snp12)$coefficients )
snp.beta = summary(result.snp12)$coefficients[2:3,1]
  print ( snp.beta )
  print ( exp(snp.beta) )
ci = confint (result.snp12)
  print (ci) 
  print ( exp(ci) )

# Allelic model #
snp.data = dbp[,c("affection", "rs1112")]
  summary(snp.data)
snp.data[,"rs1112"] <- as.numeric(snp.data[,"rs1112"]) - 1
  summary(snp.data)

# Results for allelic model (allele 2) #
result.all = glm (affection ~ rs1112, family=binomial("logit"), data=snp.data)
dev.all    = anova (result.all, test="Chi")
summary(result.all)   
print(dev.all) 


# --- Excerpt  --- #
snp.data = dbp[,c("affection", "trait","sex", "age", "rs1112", "rs1117")]
  summary(snp.data)

# Allelic model #
snp.data[,"rs1112"] <- as.numeric(snp.data[,"rs1112"]) - 1
snp.data[,"rs1117"] <- as.numeric(snp.data[,"rs1117"]) - 1


Call:  glm(formula = affection ~ rs1112, family = binomial("logit"), 
    data = dbp)

Coefficients:
(Intercept)      rs11123      rs11124  
    -0.4449       0.7582       1.5435  

Degrees of Freedom: 599 Total (i.e. Null);  597 Residual
Null Deviance:	    831.8 
Residual Deviance: 797.7 	AIC: 803.7
[1] "glm" "lm" 

Call:
glm(formula = affection ~ rs1112, family = binomial("logit"), 
    data = dbp)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -0.4449     0.1189  -3.741 0.000183 ***
rs11123       0.7582     0.1746   4.343 1.40e-05 ***
rs11124       1.5435     0.3416   4.518 6.24e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 797.75  on 597  degrees of freedom
AIC: 803.75

Number of Fisher Scoring iterations: 4

[1] 4.077856e-08
              Estimate Std. Error   z value     Pr(>|z|)
(Interc

Waiting for profiling to be done...



                 2.5 %     97.5 %
(Intercept) -0.6802726 -0.2135169
rs11123      0.4176220  1.1023701
rs11124      0.8984800  2.2475097
                2.5 %    97.5 %
(Intercept) 0.5064789 0.8077385
rs11123     1.5183466 3.0112947
rs11124     2.4558674 9.4641382


 affection rs1112 
 0:300     2:297  
 1:300     3:251  
           4: 52  

 affection     rs1112      
 0:300     Min.   :0.0000  
 1:300     1st Qu.:0.0000  
           Median :1.0000  
           Mean   :0.5917  
           3rd Qu.:1.0000  
           Max.   :2.0000  


Call:
glm(formula = affection ~ rs1112, family = binomial("logit"), 
    data = snp.data)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -0.4470     0.1142  -3.913 9.10e-05 ***
rs1112        0.7652     0.1356   5.642 1.68e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 797.75  on 598  degrees of freedom
AIC: 801.75

Number of Fisher Scoring iterations: 4


Analysis of Deviance Table

Model: binomial, link: logit

Response: affection

Terms added sequentially (first to last)


       Df Deviance Resid. Df Resid. Dev  Pr(>Chi)    
NULL                     599     831.78              
rs1112  1   34.026       598     797.75 5.438e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


 affection     trait        sex          age        rs1112  rs1117 
 0:300     Min.   : 60.50   1:329   Min.   :18.00   2:297   2:396  
 1:300     1st Qu.: 77.44   2:271   1st Qu.:38.00   3:251   3:190  
           Median : 82.00           Median :55.00   4: 52   4: 14  
           Mean   : 81.85           Mean   :55.49                  
           3rd Qu.: 86.09           3rd Qu.:74.00                  
           Max.   :101.49           Max.   :90.00                  

In [5]:
# --- II. Adjustment for the effects of covariates and of other SNPs --- #

# Covariates #
result.adj = glm (affection ~ sex + rs1112      , family=binomial("logit"), data=snp.data)
  summary(result.adj)
result.adj = glm (affection ~ age + rs1112      , family=binomial("logit"), data=snp.data)
  summary(result.adj)
result.adj = glm (affection ~ sex + age + rs1112, family=binomial("logit"), data=snp.data)
  summary(result.adj)


# Effect of SNP2 given SNP 1 # 
result.adj = glm (affection ~ rs1117 + rs1112, family=binomial("logit"), data=snp.data)
  summary(result.adj)
  anova (result.adj, test="Chi")
result.adj = glm (affection ~ rs1112 + rs1117, family=binomial("logit"), data=snp.data)
  summary(result.adj)
  anova (result.adj, test="Chi")


Call:
glm(formula = affection ~ sex + rs1112, family = binomial("logit"), 
    data = snp.data)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.08386    0.13730  -0.611    0.541    
sex2        -0.81412    0.17253  -4.719 2.37e-06 ***
rs1112       0.77139    0.13840   5.574 2.49e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 774.98  on 597  degrees of freedom
AIC: 780.98

Number of Fisher Scoring iterations: 4



Call:
glm(formula = affection ~ age + rs1112, family = binomial("logit"), 
    data = snp.data)

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.520422   0.250956  -2.074   0.0381 *  
age          0.001322   0.004020   0.329   0.7423    
rs1112       0.765189   0.135624   5.642 1.68e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 797.64  on 597  degrees of freedom
AIC: 803.64

Number of Fisher Scoring iterations: 4



Call:
glm(formula = affection ~ sex + age + rs1112, family = binomial("logit"), 
    data = snp.data)

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.198133   0.263732  -0.751    0.452    
sex2        -0.817603   0.172736  -4.733 2.21e-06 ***
age          0.002084   0.004105   0.508    0.612    
rs1112       0.771546   0.138411   5.574 2.48e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 774.72  on 596  degrees of freedom
AIC: 782.72

Number of Fisher Scoring iterations: 4



Call:
glm(formula = affection ~ rs1117 + rs1112, family = binomial("logit"), 
    data = snp.data)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -0.4523     0.1144  -3.955 7.66e-05 ***
rs1117        0.2853     0.2297   1.242  0.21431    
rs1112        0.5999     0.1883   3.186  0.00144 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 796.21  on 597  degrees of freedom
AIC: 802.21

Number of Fisher Scoring iterations: 4


Df Deviance Resid. Df Resid. Dev Pr(>Chi)    
NULL   NA       NA 599       831.7766             NA
rs1117  1 25.06376 598       806.7129   5.546551e-07
rs1112  1 10.50081 597       796.2120   1.193220e-03


Call:
glm(formula = affection ~ rs1112 + rs1117, family = binomial("logit"), 
    data = snp.data)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -0.4523     0.1144  -3.955 7.66e-05 ***
rs1112        0.5999     0.1883   3.186  0.00144 ** 
rs1117        0.2853     0.2297   1.242  0.21431    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 796.21  on 597  degrees of freedom
AIC: 802.21

Number of Fisher Scoring iterations: 4


Df Deviance  Resid. Df Resid. Dev Pr(>Chi)    
NULL   NA        NA 599       831.7766             NA
rs1112  1 34.026174 598       797.7504   5.437568e-09
rs1117  1  1.538401 597       796.2120   2.148560e-01

In [6]:
# --- III. Analysis of quantitative instead of dichotomized trait --- #

# Consideration of QT instead of binary affection status (dichotomized QT) #
result.adj = lm (trait ~ rs1112      , data=snp.data)
  summary(result.adj)
result.adj = lm (trait ~ sex + rs1112, data=snp.data)
  summary(result.adj)


Call:
lm(formula = trait ~ rs1112, data = snp.data)

Residuals:
     Min       1Q   Median       3Q      Max 
-22.5556  -3.9106   0.2194   4.0144  15.4809 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  80.1021     0.3301 242.680  < 2e-16 ***
rs1112        2.9535     0.3774   7.826 2.29e-14 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 5.954 on 598 degrees of freedom
Multiple R-squared:  0.09291,	Adjusted R-squared:  0.09139 
F-statistic: 61.25 on 1 and 598 DF,  p-value: 2.292e-14



Call:
lm(formula = trait ~ sex + rs1112, data = snp.data)

Residuals:
     Min       1Q   Median       3Q      Max 
-20.9404  -3.6272   0.2234   3.7815  16.3480 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  81.4542     0.3904 208.654  < 2e-16 ***
sex2         -2.8823     0.4748  -6.071 2.27e-09 ***
rs1112        2.8685     0.3668   7.820 2.41e-14 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 5.784 on 597 degrees of freedom
Multiple R-squared:  0.1456,	Adjusted R-squared:  0.1428 
F-statistic: 50.89 on 2 and 597 DF,  p-value: < 2.2e-16


In [7]:
# --- IV. Gene-environment (GxE) and gene-gene (GxG) interaction --- #

# GxE #
result.inter = glm (affection ~ sex * rs1112, family=binomial("logit"), data=snp.data)
  summary(result.inter)
result.inter = glm (affection ~ age * rs1112, family=binomial("logit"), data=snp.data)
  summary(result.inter)

# GxG #
result.inter = glm (affection ~ rs1112 * rs1117, family=binomial("logit"), data=snp.data)
  summary(result.inter)


Call:
glm(formula = affection ~ sex * rs1112, family = binomial("logit"), 
    data = snp.data)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.09415    0.15371  -0.613 0.540174    
sex2        -0.79026    0.23515  -3.361 0.000777 ***
rs1112       0.79049    0.18896   4.183 2.87e-05 ***
sex2:rs1112 -0.04141    0.27771  -0.149 0.881472    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 774.96  on 596  degrees of freedom
AIC: 782.96

Number of Fisher Scoring iterations: 4



Call:
glm(formula = affection ~ age * rs1112, family = binomial("logit"), 
    data = snp.data)

Coefficients:
             Estimate Std. Error z value Pr(>|z|)   
(Intercept) -0.764365   0.328207  -2.329  0.01986 * 
age          0.005719   0.005508   1.038  0.29909   
rs1112       1.193715   0.393377   3.035  0.00241 **
age:rs1112  -0.007716   0.006585  -1.172  0.24130   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 796.26  on 596  degrees of freedom
AIC: 804.26

Number of Fisher Scoring iterations: 4



Call:
glm(formula = affection ~ rs1112 * rs1117, family = binomial("logit"), 
    data = snp.data)

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -0.45855    0.11749  -3.903  9.5e-05 ***
rs1112         0.61285    0.19612   3.125  0.00178 ** 
rs1117         0.37232    0.43522   0.855  0.39228    
rs1112:rs1117 -0.07464    0.31590  -0.236  0.81323    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 796.16  on 596  degrees of freedom
AIC: 804.16

Number of Fisher Scoring iterations: 4


In [8]:
# --- V. Model selection --- #

result.reg = glm (affection ~ sex + age + rs1112 + rs1117, family=binomial("logit"), data=snp.data)
  summary(result.reg)

modelchoice.result <- step (result.reg)
  summary(modelchoice.result)


Call:
glm(formula = affection ~ sex + age + rs1112 + rs1117, family = binomial("logit"), 
    data = snp.data)

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.211378   0.264220  -0.800  0.42371    
sex2        -0.808941   0.173019  -4.675 2.93e-06 ***
age          0.002183   0.004109   0.531  0.59519    
rs1112       0.635080   0.193437   3.283  0.00103 ** 
rs1117       0.233915   0.235438   0.994  0.32045    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 773.74  on 595  degrees of freedom
AIC: 783.74

Number of Fisher Scoring iterations: 4


Start:  AIC=783.74
affection ~ sex + age + rs1112 + rs1117

         Df Deviance    AIC
- age     1   774.02 782.02
- rs1117  1   774.72 782.72
<none>        773.74 783.74
- rs1112  1   784.97 792.97
- sex     1   796.08 804.08

Step:  AIC=782.02
affection ~ sex + rs1112 + rs1117

         Df Deviance    AIC
- rs1117  1   774.98 780.98
<none>        774.02 782.02
- rs1112  1   785.34 791.34
- sex     1   796.21 802.21

Step:  AIC=780.98
affection ~ sex + rs1112

         Df Deviance    AIC
<none>        774.98 780.98
- sex     1   797.75 801.75
- rs1112  1   808.19 812.19



Call:
glm(formula = affection ~ sex + rs1112, family = binomial("logit"), 
    data = snp.data)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.08386    0.13730  -0.611    0.541    
sex2        -0.81412    0.17253  -4.719 2.37e-06 ***
rs1112       0.77139    0.13840   5.574 2.49e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 774.98  on 597  degrees of freedom
AIC: 780.98

Number of Fisher Scoring iterations: 4


In [9]:

# Data set import

load("./data/dbp.R")
ls()
dbp[1:5,]

[1] "ci"                 "dbp"                "dev.all"           
 [4] "dev.geno"           "lrt.pvalue"         "modelchoice.result"
 [7] "result.adj"         "result.all"         "result.inter"      
[10] "result.reg"         "result.snp12"       "snp.beta"          
[13] "snp.data"

fam  pid fid mid sex trait affection age rs1101 rs1102 ⋯ rs1111 rs1112
4928 4928 1   0   0   1   85.51 1         66  4      2      ⋯ 3      3     
1838 1838 1   0   0   1   84.51 1         67  2      2      ⋯ 4      2     
2450 2450 1   0   0   1   84.30 1         89  2      2      ⋯ 4      3     
647   647 1   0   0   2   89.14 1         36  4      4      ⋯ 2      3     
2772 2772 1   0   0   1   90.39 1         54  3      2      ⋯ 2      2     
     rs1113 rs1114 rs1115 rs1116 rs1117 rs1118 rs1119 rs1120
4928 3      3      3      4      2      3      3      3     
1838 2      4      4      4      2      2      4      4     
2450 3      4      3      4      2      3      3      3     
647  3      2      3      4      3      3      4      3     
2772 2      2      4      4      2      2      4      4

In [10]:
# I. Logistic regression on a single SNP genotype

result.snp12 = glm (affection ~ rs1112, family=binomial("logit"), data=dbp)
print (result.snp12)
print ( class  (result.snp12) )
print ( summary(result.snp12) )

dev.geno = anova (result.snp12, test="Chi")
lrt.pvalue = pchisq(dev.geno[dim(dev.geno)[1],"Deviance"], 
             df=2, ncp=0, FALSE) 
print ( lrt.pvalue )

print ( summary(result.snp12)$coefficients )
snp.beta = summary(result.snp12)$coefficients[2:3,1]
print ( snp.beta )
print ( exp(snp.beta) )

ci = confint (result.snp12)
print (ci) 
print ( exp(ci) )

snp.data = dbp[,c("affection", "rs1112")]
summary(snp.data)

snp.data[,"rs1112"] <- as.numeric(snp.data[,"rs1112"]) - 1
summary(snp.data)

result.all = glm (affection ~ rs1112, family=binomial("logit"), 
                  data=snp.data)
dev.all    = anova (result.all, test="Chi")
summary(result.all)   
print(dev.all) 


Call:  glm(formula = affection ~ rs1112, family = binomial("logit"), 
    data = dbp)

Coefficients:
(Intercept)      rs11123      rs11124  
    -0.4449       0.7582       1.5435  

Degrees of Freedom: 599 Total (i.e. Null);  597 Residual
Null Deviance:	    831.8 
Residual Deviance: 797.7 	AIC: 803.7
[1] "glm" "lm" 

Call:
glm(formula = affection ~ rs1112, family = binomial("logit"), 
    data = dbp)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -0.4449     0.1189  -3.741 0.000183 ***
rs11123       0.7582     0.1746   4.343 1.40e-05 ***
rs11124       1.5435     0.3416   4.518 6.24e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 797.75  on 597  degrees of freedom
AIC: 803.75

Number of Fisher Scoring iterations: 4

[1] 4.077856e-08
              Estimate Std. Error   z value     Pr(>|z|)
(Interc

Waiting for profiling to be done...



                 2.5 %     97.5 %
(Intercept) -0.6802726 -0.2135169
rs11123      0.4176220  1.1023701
rs11124      0.8984800  2.2475097
                2.5 %    97.5 %
(Intercept) 0.5064789 0.8077385
rs11123     1.5183466 3.0112947
rs11124     2.4558674 9.4641382


 affection rs1112 
 0:300     2:297  
 1:300     3:251  
           4: 52  

 affection     rs1112      
 0:300     Min.   :0.0000  
 1:300     1st Qu.:0.0000  
           Median :1.0000  
           Mean   :0.5917  
           3rd Qu.:1.0000  
           Max.   :2.0000  


Call:
glm(formula = affection ~ rs1112, family = binomial("logit"), 
    data = snp.data)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -0.4470     0.1142  -3.913 9.10e-05 ***
rs1112        0.7652     0.1356   5.642 1.68e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 797.75  on 598  degrees of freedom
AIC: 801.75

Number of Fisher Scoring iterations: 4


Analysis of Deviance Table

Model: binomial, link: logit

Response: affection

Terms added sequentially (first to last)


       Df Deviance Resid. Df Resid. Dev  Pr(>Chi)    
NULL                     599     831.78              
rs1112  1   34.026       598     797.75 5.438e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [11]:
# II. Adjustment for the effects of covariates and of other SNPs

snp.data = dbp[,c("affection", "trait","sex", "age", "rs1112", "rs1117")]
summary(snp.data)

snp.data[,"rs1112"] <- as.numeric(snp.data[,"rs1112"]) - 1
snp.data[,"rs1117"] <- as.numeric(snp.data[,"rs1117"]) - 1

result.adj = glm (affection ~ sex + rs1112      , family=binomial("logit"), 
                  data=snp.data)
summary(result.adj)

result.adj = glm (affection ~ age + rs1112      , family=binomial("logit"), 
                  data=snp.data)
summary(result.adj)

result.adj = glm (affection ~ sex + age + rs1112, family=binomial("logit"), 
                  data=snp.data)
summary(result.adj)

result.adj = glm (affection ~ rs1117 + rs1112, family=binomial("logit"), 
                  data=snp.data)
summary(result.adj)
anova (result.adj, test="Chi")

result.adj = glm (affection ~ rs1112 + rs1117, family=binomial("logit"), 
                  data=snp.data)
summary(result.adj)
anova (result.adj, test="Chi")

 affection     trait        sex          age        rs1112  rs1117 
 0:300     Min.   : 60.50   1:329   Min.   :18.00   2:297   2:396  
 1:300     1st Qu.: 77.44   2:271   1st Qu.:38.00   3:251   3:190  
           Median : 82.00           Median :55.00   4: 52   4: 14  
           Mean   : 81.85           Mean   :55.49                  
           3rd Qu.: 86.09           3rd Qu.:74.00                  
           Max.   :101.49           Max.   :90.00                  


Call:
glm(formula = affection ~ sex + rs1112, family = binomial("logit"), 
    data = snp.data)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.08386    0.13730  -0.611    0.541    
sex2        -0.81412    0.17253  -4.719 2.37e-06 ***
rs1112       0.77139    0.13840   5.574 2.49e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 774.98  on 597  degrees of freedom
AIC: 780.98

Number of Fisher Scoring iterations: 4



Call:
glm(formula = affection ~ age + rs1112, family = binomial("logit"), 
    data = snp.data)

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.520422   0.250956  -2.074   0.0381 *  
age          0.001322   0.004020   0.329   0.7423    
rs1112       0.765189   0.135624   5.642 1.68e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 797.64  on 597  degrees of freedom
AIC: 803.64

Number of Fisher Scoring iterations: 4



Call:
glm(formula = affection ~ sex + age + rs1112, family = binomial("logit"), 
    data = snp.data)

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.198133   0.263732  -0.751    0.452    
sex2        -0.817603   0.172736  -4.733 2.21e-06 ***
age          0.002084   0.004105   0.508    0.612    
rs1112       0.771546   0.138411   5.574 2.48e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 774.72  on 596  degrees of freedom
AIC: 782.72

Number of Fisher Scoring iterations: 4



Call:
glm(formula = affection ~ rs1117 + rs1112, family = binomial("logit"), 
    data = snp.data)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -0.4523     0.1144  -3.955 7.66e-05 ***
rs1117        0.2853     0.2297   1.242  0.21431    
rs1112        0.5999     0.1883   3.186  0.00144 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 796.21  on 597  degrees of freedom
AIC: 802.21

Number of Fisher Scoring iterations: 4


Df Deviance Resid. Df Resid. Dev Pr(>Chi)    
NULL   NA       NA 599       831.7766             NA
rs1117  1 25.06376 598       806.7129   5.546551e-07
rs1112  1 10.50081 597       796.2120   1.193220e-03


Call:
glm(formula = affection ~ rs1112 + rs1117, family = binomial("logit"), 
    data = snp.data)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -0.4523     0.1144  -3.955 7.66e-05 ***
rs1112        0.5999     0.1883   3.186  0.00144 ** 
rs1117        0.2853     0.2297   1.242  0.21431    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 796.21  on 597  degrees of freedom
AIC: 802.21

Number of Fisher Scoring iterations: 4


Df Deviance  Resid. Df Resid. Dev Pr(>Chi)    
NULL   NA        NA 599       831.7766             NA
rs1112  1 34.026174 598       797.7504   5.437568e-09
rs1117  1  1.538401 597       796.2120   2.148560e-01

In [12]:
# III. Analysis of quantitative instead of dichotomized trait

result.adj = lm (trait ~ rs1112      , data=snp.data)
summary(result.adj)

result.adj = lm (trait ~ sex + rs1112, data=snp.data)
summary(result.adj)



Call:
lm(formula = trait ~ rs1112, data = snp.data)

Residuals:
     Min       1Q   Median       3Q      Max 
-22.5556  -3.9106   0.2194   4.0144  15.4809 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  80.1021     0.3301 242.680  < 2e-16 ***
rs1112        2.9535     0.3774   7.826 2.29e-14 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 5.954 on 598 degrees of freedom
Multiple R-squared:  0.09291,	Adjusted R-squared:  0.09139 
F-statistic: 61.25 on 1 and 598 DF,  p-value: 2.292e-14



Call:
lm(formula = trait ~ sex + rs1112, data = snp.data)

Residuals:
     Min       1Q   Median       3Q      Max 
-20.9404  -3.6272   0.2234   3.7815  16.3480 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  81.4542     0.3904 208.654  < 2e-16 ***
sex2         -2.8823     0.4748  -6.071 2.27e-09 ***
rs1112        2.8685     0.3668   7.820 2.41e-14 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 5.784 on 597 degrees of freedom
Multiple R-squared:  0.1456,	Adjusted R-squared:  0.1428 
F-statistic: 50.89 on 2 and 597 DF,  p-value: < 2.2e-16


In [13]:
# IV. Gene-environment (GxE) and gene-gene (GxG) interaction

result.inter = glm (affection ~ sex * rs1112, family=binomial("logit"), 
                    data=snp.data)
summary(result.inter)

result.inter = glm (affection ~ age * rs1112, family=binomial("logit"), 
                    data=snp.data)
summary(result.inter)

result.inter = glm (affection ~ rs1112 * rs1117, family=binomial("logit"), 
                    data=snp.data)
summary(result.inter)


Call:
glm(formula = affection ~ sex * rs1112, family = binomial("logit"), 
    data = snp.data)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.09415    0.15371  -0.613 0.540174    
sex2        -0.79026    0.23515  -3.361 0.000777 ***
rs1112       0.79049    0.18896   4.183 2.87e-05 ***
sex2:rs1112 -0.04141    0.27771  -0.149 0.881472    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 774.96  on 596  degrees of freedom
AIC: 782.96

Number of Fisher Scoring iterations: 4



Call:
glm(formula = affection ~ age * rs1112, family = binomial("logit"), 
    data = snp.data)

Coefficients:
             Estimate Std. Error z value Pr(>|z|)   
(Intercept) -0.764365   0.328207  -2.329  0.01986 * 
age          0.005719   0.005508   1.038  0.29909   
rs1112       1.193715   0.393377   3.035  0.00241 **
age:rs1112  -0.007716   0.006585  -1.172  0.24130   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 796.26  on 596  degrees of freedom
AIC: 804.26

Number of Fisher Scoring iterations: 4



Call:
glm(formula = affection ~ rs1112 * rs1117, family = binomial("logit"), 
    data = snp.data)

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -0.45855    0.11749  -3.903  9.5e-05 ***
rs1112         0.61285    0.19612   3.125  0.00178 ** 
rs1117         0.37232    0.43522   0.855  0.39228    
rs1112:rs1117 -0.07464    0.31590  -0.236  0.81323    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 831.78  on 599  degrees of freedom
Residual deviance: 796.16  on 596  degrees of freedom
AIC: 804.16

Number of Fisher Scoring iterations: 4
